<a href="https://colab.research.google.com/github/NeelN86/AVESConn/blob/main/Copy_of_AvesConnectBackEnd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# I utilized Anvil for the front-end of this side, not required but I found success with it
!pip install anvil-uplink
import anvil.server

anvil.server.connect("specific-server-code")


In [ ]:
# import needed libraries
import pandas as pd
import numpy as np

In [ ]:
# install whatever database you want to use
url='/url-to-whatever-database-you-want-to-use.csv'
df=pd.read_csv(url)

In [ ]:
# clean up your data, in my example I filled up any null values with "None"
df['What is your current job/profession?'] = df['What is your current job/profession?'].fillna('None')
df['What are your favorite memories from your time at Sycamore?'] = df['What are your favorite memories from your time at Sycamore?'].fillna('None')


In [ ]:
# install Transformers
!pip install transformers

In [ ]:
# import Tokenizer Model to turn individual into embeddings
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

In [ ]:
# load pre-trained models, can be found online through huggingface
tokenizer=AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model=AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
# created a function to create word embeddings
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)



In [ ]:
# cleaned up data even more by comnbining name and email
df['Name']=df['Name']+' '+df['Email']

In [ ]:
# body of avesConnect function, create a dictionary for the final results
import torch
import torch.nn.functional as F
@anvil.server.callable
def avesConnect(potentialStudent):
    results = {}
    student_embeddings = []
    # tokenizes the student's inputted information and adds this to a list
    for n in potentialStudent:
        encoded_input = tokenizer(n, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            model_output = model(**encoded_input)
        embedding = mean_pooling(model_output, encoded_input['attention_mask'])
        student_embeddings.append(embedding)

    for x in range(len(df)):
        attributes = ['What did you do after high school? Be specific. \n(ex. if you went to college, where did you go, what did you major in, etc.)',
                      'What is your current job/profession?',
                      'What are your favorite memories from your time at Sycamore?']

        alumni_embeddings = []
        # tokenizes the alumni information in the dataset using the tokenizing function created earlier, then adds to these values to a list
        for attr in attributes:
            encoded_attribute = tokenizer(df[attr][x], padding=True, truncation=True, return_tensors='pt')
            with torch.no_grad():
                output = model(**encoded_attribute)
            embedding = mean_pooling(output, encoded_attribute['attention_mask'])
            alumni_embeddings.append(embedding)

        overall_similarities = []
        # utilized cosine similarity to determine how much of a match the student is with each alumni, adds each similarity value to a list
        for i in range(len(potentialStudent)):
            similarity = F.cosine_similarity(student_embeddings[i], alumni_embeddings[i], dim=1)
            overall_similarities.append(similarity)

        # Stack the cosine similarity scores along dim=1 and compute the sum for each individual
        stacked_cos_similarities = torch.stack(overall_similarities, dim=1)
        overallsimilarity = torch.sum(stacked_cos_similarities, dim=1)

        results[df['Name'][x]] = overallsimilarity.mean().item()

   # sort the results to include the 10 highest similarity scores, create a list for them, and then reveal the answer
    sortedresults = sorted(results.items(), key=lambda z: z[1], reverse=True)
    names=[]
    for a in range(10):
      names.append(sortedresults[a][0])
    return names


In [ ]:
anvil.server.wait_forever()